# Key Factor Ranking
Copyright © 2020 AA

In [1]:
%load_ext autoreload
%autoreload 2

import os, warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keyfactordiag.diag import Diag

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option("display.max_colwidth", 100)


### INIT

In [2]:
#init
diag = Diag(errorcode='4I31', modelname='DGE00')
#load Data
diag.load_mesdata()


keyparts: ['HDD' 'MB']


### Preprocess

In [3]:
#組大表
diag.getbigtable()
#將大表轉換為計算entropy要用到的資料
diag.get_analysis_table()
#output excel
#diag.output_bigtable()
#diag.output_entropy_table()

(284, 188)
12 features with greater than 0.30 missing values.

79 features with a single unique value.

Removed 84 features.
There are 50 original features
There are 227 one-hot features


**計算集中性**

In [4]:
entropy_tb_lv1 = diag.entorpy_analysis(filter_factor='All')
entropy_tb_lv2 = None

root entropy: 0.9756797868933086


**UI DEMO**

In [5]:
import ipywidgets as widgets
from IPython.display import display

In [6]:

def dropdown_factor_eventhandler(change):
    global entropy_tb_lv2
    option= change.new.split('__')[1]
    entropy_tb_lv2 = diag.entorpy_analysis(filter_factor=option)
    common_data_filter(change.new, floatSilder_faterate.value, factortype_selects.value)
        
def select_factortype_eventhandler(change):
    common_data_filter(checkbox_keyfactor.value, floatSilder_faterate.value, change.new)
    
def checkbox_keyfactor_eventhandler(change):
    common_data_filter(change.new, floatSilder_faterate.value, factortype_selects.value)
        
def floatSilder_faterate_eventhandler(change):
    common_data_filter(checkbox_keyfactor.value, change.new, factortype_selects.value)
        
def common_data_filter(key_factor, failrate, factor_types):
    factor_types = list(factortype_selects.value)
    #--- TAB 1 ---
    output_factor_lv1.clear_output()
    entropy_tb_lv1_filter = entropy_tb_lv1
    entropy_tb_lv2_filter = entropy_tb_lv2
    if key_factor:
        entropy_tb_lv1_filter = entropy_tb_lv1_filter[entropy_tb_lv1_filter['key_factor']=='1']
    entropy_tb_lv1_filter = entropy_tb_lv1_filter[entropy_tb_lv1_filter['Failrate']>=failrate]
    #if len(factor_types)>0:
    if factor_types !=['All']:
        entropy_tb_lv1_filter = entropy_tb_lv1_filter[entropy_tb_lv1_filter['Factor B'].isin(factor_types)]
        
    with output_factor_lv1:
        print(f'Case 0: ATE R5209壓件')
        display(entropy_tb_lv1_filter)     
        
    #--- TAB 2 ---
    output_factor_lv2.clear_output()        
    
    if entropy_tb_lv2_filter is None:
        return
    if key_factor:
        entropy_tb_lv2_filter = entropy_tb_lv2_filter[entropy_tb_lv2_filter['key_factor']=='1']
    entropy_tb_lv2_filter = entropy_tb_lv2_filter[entropy_tb_lv2_filter['Failrate']>=failrate]
    #if len(factor_types)>0:
    if factor_types !=['All']:
        entropy_tb_lv2_filter = entropy_tb_lv2_filter[entropy_tb_lv2_filter['Factor B'].isin(factor_types)]    
    with output_factor_lv2:
        print(f'Case 0: ATE R5209壓件')
        print(f'selected factor:{dropdown_factor.value}')
        display(entropy_tb_lv2_filter)     
    

        
    
    

****

In [7]:
item_layout = widgets.Layout(margin='0 0 50px 0')

#lv2 factor dropdown
options = list(entropy_tb_lv1['factor'].values)
options = [f'{idx}__{op}'for idx, op in enumerate(options)]
dropdown_factor = widgets.Dropdown(options = options, description='LV2 Factor', value=None)
dropdown_factor.observe(dropdown_factor_eventhandler, names='value')

factortype_options = list(entropy_tb_lv1['Factor B'].unique())
factortype_options.insert(0,'All')
factortype_selects = widgets.SelectMultiple(options=factortype_options, description='Factor Types', value=['All'])
factortype_selects.observe(select_factortype_eventhandler, names='value')

#filter
floatSilder_faterate = widgets.FloatSlider(min=0, max=1, step=0.05, description='fail rate:',value=0)
floatSilder_faterate.observe(floatSilder_faterate_eventhandler, names='value')
checkbox_keyfactor = widgets.Checkbox(description='key_factor:',value=False)
checkbox_keyfactor.observe(checkbox_keyfactor_eventhandler, names='value')


input_widgets = widgets.HBox([dropdown_factor, factortype_selects, checkbox_keyfactor, floatSilder_faterate])


#output
output_factor_lv1 = widgets.Output()
output_factor_lv2 = widgets.Output()
tab = widgets.Tab([output_factor_lv1, output_factor_lv2], layout=item_layout)
tab.set_title(0, 'main')
tab.set_title(1, 'Lv2 Table')

with output_factor_lv1:
    print(f'Case 0: ATE R5209壓件')
    display(entropy_tb_lv1)

dashboard = widgets.VBox([input_widgets, tab])



In [8]:
display(dashboard)

filter factor smtstation:ATE:STATION_NAME:ATE1
root entropy: 0.3912435636292556
filter factor smtstation:F/T:EMP_NO:1111111
root entropy: 0.08499555983671264
filter factor smtstation:F/T:STATION_NAME:F/TO2
root entropy: 0.1105808447698871
filter factor smtstation:ATE:STATION_NAME:ATE1
root entropy: 0.3912435636292556


***

In [ ]:
style = """
    <style>
       .jupyter-widgets-output-area .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
        .jupyter-widgets-output-area  {
            height: auto !important;
        }
    </style>
    """
display(widgets.HTML(style))
#wd = widgets.interact(slider, x=widgets.BoundedIntText(min=0, max=len(result_cat)-1))